In [1]:
import pandas as pd
import numpy as np
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from io import StringIO  
from IPython.display import Image  
!pip install pydotplus 
import pydotplus 
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime


# Home/Away Winner Prediction Model

In [2]:
df2 = pd.read_excel(r'/Users/markclampitt/Documents/Springboard/Capstone2/Data\new_df.xlsx')

In [3]:
df2.head()

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,Away Score_x_x,Home Score_x_x
0,-2.5,2.5,52.0,172.0,34.4,119.0,23.8,2211.0,442.2,46.6,...,3.0,1.0,3,1,0.0,5.0,1.0,5.0,NaN,NaN
1,6.5,-6.5,49.0,187.0,37.4,118.0,23.6,2484.0,496.8,49.3,...,1.0,3.0,3,1,5.0,0.0,5.0,0.0,24.0,41.0
2,6.5,-6.5,27.0,114.0,22.8,100.0,20.0,1791.0,358.2,30.9,...,11.0,99.0,99,15,16.0,11.0,21.0,11.0,24.0,27.0
3,19.5,-19.5,55.5,123.0,24.6,111.0,22.2,1998.0,399.6,38.5,...,1.0,99.0,99,3,11.0,5.0,16.0,5.0,24.0,22.0
4,31.5,-31.5,63.5,88.0,17.6,81.0,16.2,1569.0,313.8,31.4,...,99.0,99.0,99,99,25.0,13.0,35.0,18.0,45.0,21.0


In [4]:
#df2['H Score Spread'] = df2.apply(lambda x: 'H_Spread_W' if (x['Home Team Spread_x'] < 0) & (x['Away Score_x_x'] - x['Home Score_x_x'] < x['Home Team Spread_x'])  else (""),axis=1)


In [5]:


df2['H Score Spread'] = df2.apply(lambda x: 'H_Spread_W' if (x['Home Score_x_x'] + x['Home Team Spread_x']) > x['Away Score_x_x'] 
                                  else ('H_Spread_L' if (x['Home Score_x_x'] + x['Home Team Spread_x']) < x['Away Score_x_x']
                                  else 'H_Spread_Push' if x['Home Score_x_x'] + x['Home Team Spread_x'] == x['Away Score_x_x']
                                  else ""
                                       
                                       
                                       
                                       ),axis=1) 





In [51]:
# Call value_counts() on the 'Decision' column of the original coffeeData
df2['H Score Spread'].value_counts()

H_Spread_L       484
H_Spread_W       464
H_Spread_Push     28
                   1
Name: H Score Spread, dtype: int64

In [6]:
df2.sample(10)

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,Away Score_x_x,Home Score_x_x,H Score Spread
896,22.0,-22.0,45.5,95.0,19.0,97.0,19.4,1506.0,301.2,27.1,...,99.0,99,17,16.0,2.0,26.0,7.0,49.0,27.0,H_Spread_Push
632,2.5,-2.5,54.5,115.0,23.0,86.0,17.2,1877.0,375.4,29.1,...,99.0,99,25,20.0,13.0,25.0,13.0,31.0,23.0,H_Spread_L
121,14.0,-14.0,59.5,115.0,23.0,112.0,22.4,2109.0,421.8,33.2,...,99.0,99,5,20.0,8.0,30.0,8.0,51.0,35.0,H_Spread_L
146,-15.0,15.0,51.5,106.0,21.2,86.0,17.2,1739.0,347.8,27.4,...,99.0,99,99,13.0,14.0,18.0,19.0,7.0,24.0,H_Spread_W
591,-2.5,2.5,41.0,123.0,24.6,100.0,20.0,1742.0,348.4,30.7,...,9.0,13,99,6.0,14.0,6.0,14.0,12.0,17.0,H_Spread_W
135,-2.5,2.5,47.5,65.0,13.0,88.0,17.6,1598.0,319.6,30.1,...,99.0,99,99,10.0,13.0,15.0,23.0,24.0,20.0,H_Spread_L
954,-13.0,13.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,4,99,NaN,NaN,NaN,NaN,20.0,38.0,H_Spread_W
965,-7.5,7.5,50.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.0,16,99,NaN,NaN,NaN,NaN,30.0,45.0,H_Spread_W
733,-14.5,14.5,40.5,129.0,25.8,86.0,17.2,1797.0,359.4,38.4,...,1.0,1,99,0.0,15.0,0.0,17.0,0.0,36.0,H_Spread_W
157,19.5,-19.5,62.5,148.0,29.6,115.0,23.0,1917.0,383.4,40.1,...,99.0,99,2,11.0,7.0,14.0,7.0,36.0,13.0,H_Spread_L


In [7]:
df2[df2['H Score Spread'] == ""]

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,Away Score_x_x,Home Score_x_x,H Score Spread
0,-2.5,2.5,52.0,172.0,34.4,119.0,23.8,2211.0,442.2,46.6,...,1.0,3,1,0.0,5.0,1.0,5.0,NaN,NaN,


In [8]:
#y variable created below ('H/A Winner') which is whether the home or away team won. 

In [9]:
# Removed actual scores and only kept data that would be available at the start of the game
# After testing the final rf model, I noticed that it perfomed better without the conference/season records as a features

In [10]:


df2 = df2.drop('Home Score_x_x',axis =1)
df2 = df2.drop('Away Score_x_x',axis =1)
#df2 = df2.drop('Rolling 5 Past H&A Season T Losses_Away_Team',axis =1)
#df2 = df2.drop('Rolling 5 Past H&A Season T Losses_Home_Team',axis =1)
#df2 = df2.drop('Rolling 5 Past H&A Conf T Losses_Away_Team',axis =1)
#df2 = df2.drop('Rolling 5 Past H&A Conf T Losses_Home_Team',axis =1)


In [11]:
Prediction = df2[df2['H Score Spread'] == ""] #This is the championship game (Georgia ('Home')/Alabama ('Away'))
Prediction 

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Spread Avg_Away_Team,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,H Score Spread
0,-2.5,2.5,52.0,172.0,34.4,119.0,23.8,2211.0,442.2,46.6,...,-17.5,3.0,1.0,3,1,0.0,5.0,1.0,5.0,


In [12]:
df2.shape

(977, 36)

In [14]:
NoPrediction= df2.dropna()
NoPrediction = NoPrediction[1:]
NoPrediction.describe()

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Spread_Away_Team,Rolling 5 Past H&A Spread Avg_Away_Team,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team
count,928.000000,928.000000,928.000000,928.000000,928.000000,928.000000,928.000000,928.000000,928.000000,928.000000,...,928.000000,928.000000,928.000000,928.000000,928.000000,928.000000,928.000000,928.000000,928.000000,928.000000
mean,-2.789332,2.789332,51.568966,127.842672,25.568534,98.255388,19.651078,1863.168103,372.633621,36.118966,...,-2.042026,-0.408405,58.734914,57.417026,58.510776,57.703664,11.232759,11.163793,13.346983,13.302802
std,13.285394,13.285394,8.111918,42.609400,8.521880,17.716224,3.543245,378.342550,75.668510,7.391478,...,48.430751,9.686150,44.295136,44.560682,44.433919,44.457214,6.853036,6.794177,8.409957,8.275848
min,-41.500000,-36.000000,27.000000,33.000000,6.600000,47.000000,9.400000,957.000000,191.400000,18.000000,...,-148.500000,-29.700000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,-12.500000,-6.500000,45.500000,99.000000,19.800000,87.000000,17.400000,1600.750000,320.150000,31.200000,...,-33.000000,-6.600000,11.000000,10.000000,10.000000,10.000000,6.000000,5.000000,7.000000,7.000000
50%,-3.000000,3.000000,50.500000,124.000000,24.800000,97.000000,19.400000,1860.000000,372.000000,35.700000,...,-0.250000,-0.050000,99.000000,99.000000,99.000000,99.000000,11.000000,11.000000,12.000000,13.000000
75%,6.500000,12.500000,56.500000,152.000000,30.400000,110.000000,22.000000,2110.250000,422.050000,40.800000,...,34.000000,6.800000,99.000000,99.000000,99.000000,99.000000,16.000000,16.000000,19.000000,19.000000
max,36.000000,41.500000,82.500000,261.000000,52.200000,155.000000,31.000000,2987.000000,597.400000,64.300000,...,135.500000,27.100000,99.000000,99.000000,99.000000,99.000000,35.000000,31.000000,40.000000,40.000000


In [15]:
columns= NoPrediction.columns


In [16]:
NoPrediction.head()

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Spread Avg_Away_Team,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,H Score Spread
1,6.5,-6.5,49.0,187.0,37.4,118.0,23.6,2484.0,496.8,49.3,...,-21.9,1.0,3.0,3,1,5.0,0.0,5.0,0.0,H_Spread_W
2,6.5,-6.5,27.0,114.0,22.8,100.0,20.0,1791.0,358.2,30.9,...,-3.7,11.0,99.0,99,15,16.0,11.0,21.0,11.0,H_Spread_W
3,19.5,-19.5,55.5,123.0,24.6,111.0,22.2,1998.0,399.6,38.5,...,-22.1,1.0,99.0,99,3,11.0,5.0,16.0,5.0,H_Spread_W
4,31.5,-31.5,63.5,88.0,17.6,81.0,16.2,1569.0,313.8,31.4,...,6.8,99.0,99.0,99,99,25.0,13.0,35.0,18.0,H_Spread_W
5,-14.5,14.5,63.0,156.0,31.2,125.0,25.0,2280.0,456.0,40.4,...,9.2,99.0,21.0,25,99,15.0,18.0,15.0,23.0,H_Spread_W


In [18]:
NoPrediction[NoPrediction['H Score Spread'] ==""]

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Spread Avg_Away_Team,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,H Score Spread


In [19]:
#Feature Selection


features = []
for i in columns:
    features.append(i)
features.remove('H Score Spread')



X = NoPrediction[features] 

y = NoPrediction['H Score Spread']

# Model 1 - entr tree.DecisionTreeClassifier

In [20]:
# Call train_test_split on X, y. Make the test_size = 0.25, and random_state = 246
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.25,random_state=246)

In [21]:
# Declare a variable called entr_model and use tree.DecisionTreeClassifier. 
entr_model= tree.DecisionTreeClassifier(criterion="entropy",random_state = 1234)
# Call fit() on entr_model
entr_model.fit(X_train,y_train)

# Call predict() on entr_model with X_test passed to it, and assign the result to a variable y_pred 
y_pred = entr_model.predict(X_test)

# Call Series on our y_pred variable with the following: pd.Series(y_pred)
y_pred = pd.Series(y_pred)

# Check out entr_model
entr_model

DecisionTreeClassifier(criterion='entropy', random_state=1234)

In [22]:
# As before, we need to visualize the tree to grasp its nature
#dot_data = StringIO()
#tree.export_graphviz(entr_model, out_file=dot_data,  
              #  filled=True, rounded=True,
              #  special_characters=True, feature_names=X_train.columns,class_names = ["Away", "Home"])

# Alternatively for class_names use entr_model2.classes_
#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
#Image(graph.create_png())

In [41]:
# Run this block for model evaluation metrics 
print("Model Entropy - no max depth")

#metrics.confusion_matrix(y_test, y_pred)
print(metrics.classification_report(y_test, y_pred, digits=3))

#print("Accuracy:", metrics.accuracy_score(y_test,y_pred))
#print("Balanced accuracy:", metrics.balanced_accuracy_score(y_test,y_pred))
#print('Precision score for "Home"' , metrics.precision_score(y_test,y_pred, average='micro'))
#print('Precision score for "Away"' , metrics.precision_score(y_test,y_pred,average='micro' ))
#print('Recall score for "Home"' , metrics.recall_score(y_test,y_pred, average='micro'))
#print('Recall score for "Away"' , metrics.recall_score(y_test,y_pred, average='micro'))

Model Entropy - no max depth
               precision    recall  f1-score   support

   H_Spread_L      0.550     0.569     0.559       116
H_Spread_Push      0.000     0.000     0.000         6
   H_Spread_W      0.553     0.518     0.535       110

     accuracy                          0.530       232
    macro avg      0.368     0.362     0.365       232
 weighted avg      0.537     0.530     0.533       232



# Model 2 gini tree.DecisionTreeClassifier

In [42]:
# Make a variable called gini_model, and assign it exactly what you assigned entr_model with above, but with the
# criterion changed to 'gini'
gini_model= tree.DecisionTreeClassifier(criterion="gini",random_state = 1234)
# Call fit() on the gini_model as you did with the entr_model
gini_model.fit(X_train,y_train)

# Call predict() on the gini_model as you did with the entr_model 
y_pred = gini_model.predict(X_test)

# Turn y_pred into a series, as before
y_pred = pd.Series(y_pred)

# Check out gini_model
gini_model

DecisionTreeClassifier(random_state=1234)

In [43]:
# As before, but make the model name gini_model

#dot_data = StringIO()
#tree.export_graphviz(gini_model, out_file=dot_data,  
               # filled=True, rounded=True,
               # special_characters=True, feature_names=X_train.columns,class_names = ["Away", "Home"])
# Alternatively for class_names use gini_model.classes_
#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
#Image(graph.create_png())

In [44]:
# Run this block for model evaluation
print(metrics.classification_report(y_test, y_pred, digits=3))
#print("Model Gini impurity model")
#print("Accuracy:", metrics.accuracy_score(y_test,y_pred))
#print("Balanced accuracy:", metrics.balanced_accuracy_score(y_test,y_pred))
#print('Precision score' , metrics.precision_score(y_test,y_pred, average='micro'))
#print('Recall score' , metrics.recall_score(y_test,y_pred, average='micro'))

               precision    recall  f1-score   support

   H_Spread_L      0.550     0.569     0.559       116
H_Spread_Push      0.000     0.000     0.000         6
   H_Spread_W      0.553     0.518     0.535       110

     accuracy                          0.530       232
    macro avg      0.368     0.362     0.365       232
 weighted avg      0.537     0.530     0.533       232



# Model 3 - entr tree.DecisionTreeClassifier w/ max depth 3

In [45]:
# Made a model as before, but call it entr_model2, and make the max_depth parameter equal to 3. 
# Execute the fitting, predicting, and Series operations as before
entr_model2 = tree.DecisionTreeClassifier(criterion="entropy",random_state = 1234,max_depth = 3)
entr_model2.fit(X_train,y_train)
y_pred = entr_model2.predict(X_test)
y_pred = pd.Series(y_pred)
entr_model2


DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=1234)

In [46]:
# As before, we need to visualize the tree to grasp its nature

#dot_data = StringIO()
#tree.export_graphviz(entr_model2, out_file=dot_data,  
               # filled=True, rounded=True,
               # special_characters=True, feature_names=X_train.columns,class_names = ["Away", "Home"])
# Alternatively for class_names use entr_model2.classes_

#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
#Image(graph.create_png())

In [47]:
# Run this block for model evaluation 
print(metrics.classification_report(y_test, y_pred, digits=3))
#print("Model Entropy model max depth 3")
#print("Accuracy:", metrics.accuracy_score(y_test,y_pred))
#print("Balanced accuracy:", metrics.balanced_accuracy_score(y_test,y_pred))
#print('Precision score for "Home"' , metrics.precision_score(y_test,y_pred, pos_label = "H_Spread_W"))
#print('Recall score for "Away"' , metrics.recall_score(y_test,y_pred, pos_label = "H_Spread_L"))

               precision    recall  f1-score   support

   H_Spread_L      0.494     0.362     0.418       116
H_Spread_Push      0.000     0.000     0.000         6
   H_Spread_W      0.476     0.636     0.545       110

     accuracy                          0.483       232
    macro avg      0.323     0.333     0.321       232
 weighted avg      0.473     0.483     0.467       232



/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Model 4 - gini - tree.DecisionTreeClassifier Max depth 3

In [48]:
# As before, make a variable, but call it gini_model2, and ensure the max_depth parameter is set to 3
gini_model2 = tree.DecisionTreeClassifier(criterion ='gini', random_state = 1234, max_depth  = 3 )

# Do the fit, predict, and series transformations as before. 
gini_model2.fit(X_train,y_train)
y_pred = gini_model2.predict(X_test)
y_pred = pd.Series(y_pred)
gini_model2

DecisionTreeClassifier(max_depth=3, random_state=1234)

In [49]:
#dot_data = StringIO()
#tree.export_graphviz(gini_model2, out_file=dot_data,  
            #    filled=True, rounded=True,
             #   special_characters=True, feature_names=X_train.columns,class_names = ["Away", "Home"])
# Alternatively for class_names use gini_model2.classes_

#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
#Image(graph.create_png())

In [50]:
print(metrics.classification_report(y_test, y_pred, digits=3))
#print("Gini impurity  model - max depth 3")
#print("Accuracy:", metrics.accuracy_score(y_test,y_pred))
#print("Balanced accuracy:", metrics.balanced_accuracy_score(y_test,y_pred))
#print('Precision score' , metrics.precision_score(y_test,y_pred, pos_label = "H_Spread_W"))
#print('Recall score' , metrics.recall_score(y_test,y_pred, pos_label = "H_Spread_L"))

               precision    recall  f1-score   support

   H_Spread_L      0.512     0.931     0.661       116
H_Spread_Push      0.000     0.000     0.000         6
   H_Spread_W      0.619     0.118     0.198       110

     accuracy                          0.522       232
    macro avg      0.377     0.350     0.286       232
 weighted avg      0.549     0.522     0.424       232



/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:

feature_cols = features


new_X = Prediction[feature_cols]

In [53]:
# Call get_dummies() on the Pandas object pd, with new_X plugged in, to one-hot encode all features in the training set
new_X = pd.get_dummies(new_X)

# Make a variable called potential_buyers, and assign it the result of calling predict() on a model of your choice; 
# don't forget to pass new_X to predict()

Home_Wins = gini_model2.predict(new_X)

In [54]:
np.unique(Home_Wins, return_counts=True)

(array(['H_Spread_W'], dtype=object), array([1]))

# Model 5 - RF /entr / max_depth =2

In [55]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [64]:
# Plug in appropriate max_depth and random_state parameters 
firstRFModel = RandomForestClassifier(max_depth= 3, random_state= 1234, criterion = 'entropy',)
firstRFModel.fit(X_train, y_train)


# Model and fit
y_pred = firstRFModel.predict(X_test)
y_pred = pd.Series(y_pred)


In [65]:
print(metrics.classification_report(y_test, y_pred, digits=3))
#print("RF  model - max depth 3")
#print("Accuracy:", metrics.accuracy_score(y_test,y_pred))
#print("Balanced accuracy:", metrics.balanced_accuracy_score(y_test,y_pred))
#print('Precision score' , metrics.precision_score(y_test,y_pred, pos_label = "H_Spread_W"))
#print('Recall score' , metrics.recall_score(y_test,y_pred, pos_label = "H_Spread_L"))

               precision    recall  f1-score   support

   H_Spread_L      0.517     0.664     0.581       116
H_Spread_Push      0.000     0.000     0.000         6
   H_Spread_W      0.506     0.382     0.435       110

     accuracy                          0.513       232
    macro avg      0.341     0.349     0.339       232
 weighted avg      0.498     0.513     0.497       232



/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [58]:
Prediction

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Spread Avg_Away_Team,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,H Score Spread
0,-2.5,2.5,52.0,172.0,34.4,119.0,23.8,2211.0,442.2,46.6,...,-17.5,3.0,1.0,3,1,0.0,5.0,1.0,5.0,


In [59]:
X = Prediction.drop('H Score Spread',axis =1)
X

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Spread_Away_Team,Rolling 5 Past H&A Spread Avg_Away_Team,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team
0,-2.5,2.5,52.0,172.0,34.4,119.0,23.8,2211.0,442.2,46.6,...,-87.5,-17.5,3.0,1.0,3,1,0.0,5.0,1.0,5.0


In [60]:
firstRFModel.predict(X) #Predicted Georgia to Win

array(['H_Spread_W'], dtype=object)

In [61]:
firstRFModel.predict_proba(X) #Slight edge given to Georgia, with 53% chance of winning.

array([[0.43812731, 0.02839605, 0.53347665]])

In [62]:
#Plug in appropriate max_depth and random_state parameters 
#secondRFModel = RandomForestClassifier(random_state= 1234)
#secondRFModel.fit(X_train, y_train)


 #Model and fit
#y_pred = secondRFModel.predict(X_test)
#y_pred = pd.Series(y_pred)


In [63]:
#print("RF2  model - max depth 3")
#print("Accuracy:", metrics.accuracy_score(y_test,y_pred))
#print("Balanced accuracy:", metrics.balanced_accuracy_score(y_test,y_pred))
#print('Precision score' , metrics.precision_score(y_test,y_pred, pos_label = "Home"))
#print('Recall score' , metrics.recall_score(y_test,y_pred, pos_label = "Away"))